In [76]:
import pandas as pd
import xml.etree.ElementTree as ET

In [77]:
df = pd.read_csv("data/sub-data.txt")

In [78]:
df.head()

,CASEID,YEAR,ID,AGE,SEX,RACE,RACECEN1,RACECEN2,RACECEN3,HISPANIC,...,LIFE,HELPFUL,FAIR,TRUST,TRUSTY,SATCITY,SATHOBBY,SATFAM,SATFRND,SATHEALT
0,1,1972,1,23,2,1,0,0,0,0,...,0,2,2,3,0,0,0,0,0,0
1,2,1972,2,70,1,1,0,0,0,0,...,0,1,2,1,0,0,0,0,0,0
2,3,1972,3,48,2,1,0,0,0,0,...,0,2,1,2,0,0,0,0,0,0
3,4,1972,4,27,2,1,0,0,0,0,...,0,2,2,2,0,0,0,0,0,0
4,5,1972,5,61,2,1,0,0,0,0,...,0,2,2,2,0,0,0,0,0,0


In [79]:
root = ET.parse('data/sub-ddi.txt').getroot()

data = root.find("dataDscr")

var_dict = {}
for var in data:
    # Get the ID Code of the column
    id_code = var.get("ID")
    # Get the Full Name of the column
    full_title = var.find("./labl").text.strip()
    # Get the Question Text (if applicable)
    qstn = var.find("qstn")
    if qstn != None:
        qlit = qstn.find("qstnLit").text
        question_text = qlit.strip()#.replace("\n", "")
    else:
        question_text = None
    
    # Get the categories
    category_dict = {}
    category_list = var.findall("catgry")
    for category in category_list:
        value = float(category.find("catValu").text.strip())
        label = category.find("*[@level='category']").text.strip()
        missing = category.get("missing")
        category_dict[value] = label
#         category_dict[value] = {"label":label, "missing":missing}
    
    # Compile into dictionary
    var_dict[id_code] = {
        "title": full_title,
        "question_text": question_text,
        "categories": category_dict
    }

In [82]:
value_df = df.copy()

for col in value_df.columns:
    try:
        value_df[col].replace(to_replace=var_dict[col]['categories'], inplace=True)
    except AttributeError as e:
        print(e)
        print(var)

In [83]:
value_df.head()

,CASEID,YEAR,ID,AGE,SEX,RACE,RACECEN1,RACECEN2,RACECEN3,HISPANIC,...,LIFE,HELPFUL,FAIR,TRUST,TRUSTY,SATCITY,SATHOBBY,SATFAM,SATFRND,SATHEALT
0,1,1972,1,23,FEMALE,WHITE,IAP,IAP,IAP,IAP,...,IAP,LOOKOUT FOR SELF,FAIR,DEPENDS,IAP,IAP,IAP,IAP,IAP,IAP
1,2,1972,2,70,MALE,WHITE,IAP,IAP,IAP,IAP,...,IAP,HELPFUL,FAIR,CAN TRUST,IAP,IAP,IAP,IAP,IAP,IAP
2,3,1972,3,48,FEMALE,WHITE,IAP,IAP,IAP,IAP,...,IAP,LOOKOUT FOR SELF,TAKE ADVANTAGE,CANNOT TRUST,IAP,IAP,IAP,IAP,IAP,IAP
3,4,1972,4,27,FEMALE,WHITE,IAP,IAP,IAP,IAP,...,IAP,LOOKOUT FOR SELF,FAIR,CANNOT TRUST,IAP,IAP,IAP,IAP,IAP,IAP
4,5,1972,5,61,FEMALE,WHITE,IAP,IAP,IAP,IAP,...,IAP,LOOKOUT FOR SELF,FAIR,CANNOT TRUST,IAP,IAP,IAP,IAP,IAP,IAP


In [69]:
for var in var_dict:
    cat_dict = var_dict[var]['categories']
    cat_set = set()
    for cat in cat_dict:
        label = cat_dict[cat]['label']
        if label in cat_set:
            print("ERROR: VAR={}, Cat={}, Label={}".format(var, cat, label))
        else:
            cat_set.add(label)

ERROR: VAR=HHTYPE, Cat=26, Label=3ADLTS,0MAR,REL,0KIDS
ERROR: VAR=HHTYPE, Cat=46, Label=3ADLTS,2MAR,REL,NTREL,0KIDS
ERROR: VAR=HHTYPE, Cat=52, Label=3ADLTS,3MAR,SP,REL,0KIDS
ERROR: VAR=HHTYPE, Cat=126, Label=3ADLTS,0MAR,REL,1+KIDS
ERROR: VAR=HHTYPE, Cat=146, Label=3ADLTS,2MAR,REL,NTREL,1+KIDS
ERROR: VAR=HHTYPE, Cat=152, Label=3ADLTS,3MAR,SP,REL,1+KIDS
ERROR: VAR=PAIND16, Cat=999, Label=NA
ERROR: VAR=OTHER, Cat=165, Label=Church of the Living God
ERROR: VAR=OTHER, Cat=209, Label=Church of the Living God
ERROR: VAR=OTH16, Cat=165, Label=Church of the Living God
ERROR: VAR=OTH16, Cat=209, Label=Church of the Living God
ERROR: VAR=SPOTHER, Cat=165, Label=Church of the Living God
ERROR: VAR=SPOTHER, Cat=209, Label=Church of the Living God
ERROR: VAR=SPOTH16, Cat=165, Label=Church of the Living God
